In [108]:
import pandas as pd
import numpy as np



#######################
####데이터 읽어오기####
#######################
train_X = pd.read_csv("https://raw.githubusercontent.com/Datamanim/jeju/main/Jeju_trainX.csv",encoding='euc-kr')
train_y= pd.read_csv("https://raw.githubusercontent.com/Datamanim/jeju/main/Jeju_trainy.csv",encoding='euc-kr')
test_X= pd.read_csv("https://raw.githubusercontent.com/Datamanim/jeju/main/Jeju_testX.csv",encoding='euc-kr')
sub= pd.read_csv("https://raw.githubusercontent.com/Datamanim/jeju/main/subExample.csv",encoding='euc-kr')
#print(train_X.head(3))
#print(train_y.head(3))
#print(test_X.head(3))
#print(sub.head(3))



##################################################
####결측치수, 데이터타입, 연속형변수 분포 확인####
##################################################
#print(train_X.info())
#print(test_X.info())
#print(train_y.info())
#print(sub.info())
#print(train_X.describe())
#print(test_X.describe())

#corr_df=pd.merge(train_X, train_y, on='id')
#corr_df=corr_df[['거주인구', '근무인구', '방문인구', '총 유동인구', '평균 속도', 
#                 '평균 소요 시간', '평균 기온', '일강수량', '평균 풍속','교통량']]
#print(corr_df.corr()['교통량'])




# 이상치 기준
total=pd.concat([train_X, test_X], axis=0)
#print(total.describe())
total_num=total[['거주인구', '근무인구', '방문인구', '총 유동인구', '평균 속도', 
                 '평균 소요 시간', '평균 기온', '일강수량', '평균 풍속']]
total_str=total[['일자', '시도명', '읍면동명']]
lower_outlier=list(map(lambda x: np.percentile(total_num[x],25)-((np.percentile(total_num[x],75)-np.percentile(total_num[x],25))*1.5) ,total_num.columns))
upper_outlier=list(map(lambda x: np.percentile(total_num[x],75)+((np.percentile(total_num[x],75)-np.percentile(total_num[x],25))*1.5) ,total_num.columns))
#print(lower_outlier)
#print(upper_outlier)
# 정규화
from sklearn.preprocessing import StandardScaler
standardscaler=StandardScaler()
standardscaler.fit(total_num)


# 범주형변수 train-test비교 - 차이없음
#print(set(train_X['시도명'])-set(test_X['시도명']))
#print(set(test_X['시도명'])-set(train_X['시도명']))
#print(set(train_X['읍면동명'])-set(test_X['읍면동명']))
#print(set(test_X['읍면동명'])-set(train_X['읍면동명']))
#print(train_X['시도명'].value_counts())
#print(test_X['시도명'].value_counts())
#print(train_X['읍면동명'].value_counts())
#print(test_X['읍면동명'].value_counts())



def preprocessing(xdata):
    x_num=xdata[['거주인구', '근무인구', '방문인구', '총 유동인구', '평균 속도', 
                 '평균 소요 시간', '평균 기온', '일강수량', '평균 풍속']]
    x_str=xdata[['일자', '시도명', '읍면동명']]
    
    #for i in range(len(x_num.columns)):
    #    x_num[x_num.columns[i]]=[lower_outlier[i] if x<lower_outlier[i] else upper_outlier[i] if x>upper_outlier[i] else x for x in x_num[x_num.columns[i]]]
    #print(x_num.describe())
    x_num=pd.DataFrame(standardscaler.transform(x_num), columns=x_num.columns)
    #print(x_num.describe())
    x_str=pd.get_dummies(x_str, columns=['시도명','읍면동명'])
    x_str['일자']=pd.to_datetime(x_str['일자'])    
    x_str['연도']=x_str['일자'].dt.year
    x_str['월']=x_str['일자'].dt.month
    x_str['일']=x_str['일자'].dt.day
    x_str['weekend']=x_str['일자'].dt.weekday
    x_str.drop('일자', axis=1, inplace=True)
    #print(x_str)
    x_data=pd.concat([x_num, x_str], axis=1)
    #x_data.drop(['평균 속도', '평균 소요 시간', '평균 기온', '일강수량', '평균 풍속'], axis=1, inplace=True)
    return x_data
    
x_data=preprocessing(train_X)
corr_df=pd.concat([x_data, train_y], axis=1)
corr_df=corr_df.drop('id', axis=1)
#print(corr_df.corr()['교통량'].)
corr_prep=corr_df.corr()['교통량']
#print(corr_prep[abs(corr_prep)<0.1].index)
#print(corr_prep[abs(corr_prep)>=0.3])
corr_del_index=corr_prep[abs(corr_prep)<0.1].index
x_data.drop(corr_del_index, axis=1, inplace=True)

#corr_inc_index=['거주인구','근무인구','총 유동인구','평균 속도','시도명_서귀포시','시도명_제주시','읍면동명_노형동','읍면동명_도두동','읍면동명_화북동']

from sklearn.decomposition import PCA
#pca=PCA()
#pca.fit(x_data)
#print(pca.explained_variance_ratio_.cumsum())
pca=PCA(n_components=20)
pca.fit(x_data)
train_data=pd.DataFrame(pca.transform(x_data))
#train_data=pd.concat([train_data, x_data[corr_inc_index]], axis=1)
#print(train_data)




#print(train_data.head(5))
#print(train_y.head(5))
from sklearn.model_selection import train_test_split
x_train, x_val,  y_train, y_val = train_test_split(train_data, train_y['교통량'], test_size=0.2, random_state=42)
#print(x_train)
#print(y_train)
#print(len(x_val))
#print(len(y_val))

from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
'''
from sklearn.linear_model import LinearRegression
linear=LinearRegression()
linear.fit(x_train, y_train)
print(mean_squared_error(linear.predict(x_train),y_train))
print(mean_squared_error(linear.predict(x_val),y_val))


from sklearn.tree import DecisionTreeRegressor
tree=DecisionTreeRegressor()
tree.fit(x_train, y_train)
print(mean_squared_error(tree.predict(x_train), y_train))
print(mean_squared_error(tree.predict(x_val), y_val))

from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor()
rf.fit(x_train, y_train)
print(mean_squared_error(rf.predict(x_train), y_train))
print(mean_squared_error(rf.predict(x_val), y_val))

from sklearn.ensemble import GradientBoostingRegressor
gb=GradientBoostingRegressor()
gb.fit(x_train, y_train)
print(mean_squared_error(gb.predict(x_train), y_train))
print(mean_squared_error(gb.predict(x_val), y_val))
'''

###하이퍼파라미터

from sklearn.model_selection import GridSearchCV
param_grid = [
    {
        'n_estimators':[3,10,30], 'max_features':[2,4,6,8]
    },
    {
        'bootstrap':[False], 'n_estimators':[3,10], 'max_features':[2,3,4]
    },
]
#grid_rf=GridSearchCV(RandomForestRegressor(), param_grid, cv=5, scoring='neg_mean_squared_error')
#grid_rf.fit(x_train, y_train)


grid_rf=RandomForestRegressor(max_features=8, n_estimators=30)
grid_rf.fit(x_train, y_train)
#print(grid_rf.best_params_)
#print(grid_rf.best_score_)
print(mean_squared_error(grid_rf.predict(x_train), y_train))
print(mean_squared_error(grid_rf.predict(x_val), y_val))

test_x_data=preprocessing(test_X)
test_x_data.drop(corr_del_index, axis=1, inplace=True)
test_data=pd.DataFrame(pca.transform(test_x_data))
sub['교통량']=grid_rf.predict(test_data)

1572.4168027209098
10369.467107156093


In [112]:
import pandas as pd
import numpy as np



#######################
####데이터 읽어오기####
#######################
train_X = pd.read_csv("https://raw.githubusercontent.com/Datamanim/jeju/main/Jeju_trainX.csv",encoding='euc-kr')
train_y= pd.read_csv("https://raw.githubusercontent.com/Datamanim/jeju/main/Jeju_trainy.csv",encoding='euc-kr')
test_X= pd.read_csv("https://raw.githubusercontent.com/Datamanim/jeju/main/Jeju_testX.csv",encoding='euc-kr')
sub= pd.read_csv("https://raw.githubusercontent.com/Datamanim/jeju/main/subExample.csv",encoding='euc-kr')
#print(train_X.head(3))
#print(train_y.head(3))
#print(test_X.head(3))
#print(sub.head(3))



##################################################
####결측치수, 데이터타입, 연속형변수 분포 확인####
##################################################
#print(train_X.info())
#print(test_X.info())
#print(train_y.info())
#print(sub.info())
#print(train_X.describe())
#print(test_X.describe())

#corr_df=pd.merge(train_X, train_y, on='id')
#corr_df=corr_df[['거주인구', '근무인구', '방문인구', '총 유동인구', '평균 속도', 
#                 '평균 소요 시간', '평균 기온', '일강수량', '평균 풍속','교통량']]
#print(corr_df.corr()['교통량'])




# 이상치 기준
total=pd.concat([train_X, test_X], axis=0)
#print(total.describe())
total_num=total[['거주인구', '근무인구', '방문인구', '총 유동인구', '평균 속도', 
                 '평균 소요 시간', '평균 기온', '일강수량', '평균 풍속']]
total_str=total[['일자', '시도명', '읍면동명']]
lower_outlier=list(map(lambda x: np.percentile(total_num[x],25)-((np.percentile(total_num[x],75)-np.percentile(total_num[x],25))*1.5) ,total_num.columns))
upper_outlier=list(map(lambda x: np.percentile(total_num[x],75)+((np.percentile(total_num[x],75)-np.percentile(total_num[x],25))*1.5) ,total_num.columns))
#print(lower_outlier)
#print(upper_outlier)
# 정규화
from sklearn.preprocessing import RobustScaler
robustscaler=RobustScaler()
robustscaler.fit(total_num)


# 범주형변수 train-test비교 - 차이없음
#print(set(train_X['시도명'])-set(test_X['시도명']))
#print(set(test_X['시도명'])-set(train_X['시도명']))
#print(set(train_X['읍면동명'])-set(test_X['읍면동명']))
#print(set(test_X['읍면동명'])-set(train_X['읍면동명']))
#print(train_X['시도명'].value_counts())
#print(test_X['시도명'].value_counts())
#print(train_X['읍면동명'].value_counts())
#print(test_X['읍면동명'].value_counts())



def preprocessing(xdata):
    x_num=xdata[['거주인구', '근무인구', '방문인구', '총 유동인구', '평균 속도', 
                 '평균 소요 시간', '평균 기온', '일강수량', '평균 풍속']]
    x_str=xdata[['일자', '시도명', '읍면동명']]
    
    #for i in range(len(x_num.columns)):
    #    x_num[x_num.columns[i]]=[lower_outlier[i] if x<lower_outlier[i] else upper_outlier[i] if x>upper_outlier[i] else x for x in x_num[x_num.columns[i]]]
    #print(x_num.describe())
    x_num=pd.DataFrame(robustscaler.transform(x_num), columns=x_num.columns)
    #print(x_num.describe())
    x_str=pd.get_dummies(x_str, columns=['시도명','읍면동명'])
    x_str['일자']=pd.to_datetime(x_str['일자'])    
    x_str['연도']=x_str['일자'].dt.year
    x_str['월']=x_str['일자'].dt.month
    x_str['일']=x_str['일자'].dt.day
    x_str['weekend']=x_str['일자'].dt.weekday
    x_str.drop('일자', axis=1, inplace=True)
    #print(x_str)
    x_data=pd.concat([x_num, x_str], axis=1)
    #x_data.drop(['평균 속도', '평균 소요 시간', '평균 기온', '일강수량', '평균 풍속'], axis=1, inplace=True)
    return x_data
    
x_data=preprocessing(train_X)
corr_df=pd.concat([x_data, train_y], axis=1)
corr_df=corr_df.drop('id', axis=1)
#print(corr_df.corr()['교통량'].)
corr_prep=corr_df.corr()['교통량']
#print(corr_prep[abs(corr_prep)<0.1].index)
#print(corr_prep[abs(corr_prep)>=0.3])
corr_del_index=corr_prep[abs(corr_prep)<0.1].index
x_data.drop(corr_del_index, axis=1, inplace=True)

#corr_inc_index=['거주인구','근무인구','총 유동인구','평균 속도','시도명_서귀포시','시도명_제주시','읍면동명_노형동','읍면동명_도두동','읍면동명_화북동']

from sklearn.decomposition import PCA
pca=PCA()
pca.fit(x_data)
print(pca.explained_variance_ratio_.cumsum())
pca=PCA(n_components=18)
pca.fit(x_data)
train_data=pd.DataFrame(pca.transform(x_data))
#train_data=pd.concat([train_data, x_data[corr_inc_index]], axis=1)
#print(train_data)




#print(train_data.head(5))
#print(train_y.head(5))
from sklearn.model_selection import train_test_split
x_train, x_val,  y_train, y_val = train_test_split(train_data, train_y['교통량'], test_size=0.2, random_state=42)
#print(x_train)
#print(y_train)
#print(len(x_val))
#print(len(y_val))

from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

from sklearn.linear_model import LinearRegression
linear=LinearRegression()
linear.fit(x_train, y_train)
print(mean_squared_error(linear.predict(x_train),y_train))
print(mean_squared_error(linear.predict(x_val),y_val))


from sklearn.tree import DecisionTreeRegressor
tree=DecisionTreeRegressor()
tree.fit(x_train, y_train)
print(mean_squared_error(tree.predict(x_train), y_train))
print(mean_squared_error(tree.predict(x_val), y_val))

from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor()
rf.fit(x_train, y_train)
print(mean_squared_error(rf.predict(x_train), y_train))
print(mean_squared_error(rf.predict(x_val), y_val))

from sklearn.ensemble import GradientBoostingRegressor
gb=GradientBoostingRegressor()
gb.fit(x_train, y_train)
print(mean_squared_error(gb.predict(x_train), y_train))
print(mean_squared_error(gb.predict(x_val), y_val))


###하이퍼파라미터

from sklearn.model_selection import GridSearchCV
param_grid = [
    {
        'n_estimators':[3,10,30], 'max_features':[2,4,6,8]
    },
    {
        'bootstrap':[False], 'n_estimators':[3,10], 'max_features':[2,3,4]
    },
]
#grid_rf=GridSearchCV(RandomForestRegressor(), param_grid, cv=5, scoring='neg_mean_squared_error')
#grid_rf.fit(x_train, y_train)


grid_rf=RandomForestRegressor(max_features=8, n_estimators=30)
grid_rf.fit(x_train, y_train)
#print(grid_rf.best_params_)
#print(grid_rf.best_score_)
print(mean_squared_error(grid_rf.predict(x_train), y_train))
print(mean_squared_error(grid_rf.predict(x_val), y_val))

test_x_data=preprocessing(test_X)
test_x_data.drop(corr_del_index, axis=1, inplace=True)
test_data=pd.DataFrame(pca.transform(test_x_data))
sub['교통량']=grid_rf.predict(test_data)

[0.72059025 0.80798509 0.85381721 0.88174028 0.90344341 0.90710426
 0.9107651  0.91442595 0.9180868  0.92174703 0.92540694 0.92906592
 0.93272405 0.9363814  0.94003615 0.94369048 0.94734482 0.95099915
 0.95465348 0.95830781 0.96196215 0.96561648 0.96927081 0.97292514
 0.97657948 0.98022896 0.98386584 0.98749124 0.99111135 0.99394915
 0.99615709 0.99744315 0.9985538  0.99937786 1.         1.
 1.        ]
110449.78903716497
116318.78331261258
0.0
18559.6396985539
1501.382937836535
10832.31712436055
15315.754085685541
17551.906036366512
1624.5008860439652
10537.501906314672
